In [5]:
# set your environment variables
SA_FILE="felipe-sandbox-ba2c6045e3e1.json" # upload service account file
PROJECT_ID="felipe-sandbox"
APP_NAME="pytorch-test"
REGION="us-central1"

In [44]:
# copy environment variables on terminal
import os
os.environ['SA_FILE']=SA_FILE
os.environ['PROJECT_ID']=PROJECT_ID
os.environ['APP_NAME']=APP_NAME
os.environ['REGION']=REGION
os.environ['IMAGE_NAME']=REGION+"-docker.pkg.dev/"+PROJECT_ID+"/"+APP_NAME+"/serve-mnist"

In [ ]:
!echo $IMAGE_NAME

In [23]:
%%bash
# authenticate notebook
gcloud auth activate-service-account --key-file="$SA_FILE"
gcloud config set project "${PROJECT_ID}"

Activated service account credentials for: [pytorch-test@felipe-sandbox.iam.gserviceaccount.com]
Updated property [core/project].


Enable APIs: 
- Cloud Build API
- Cloud Run Admin API
- Container Registry API
- Cloud Resource Manager API

Roles
- Vertex AI User
- Service Usage Viewer
- Storage Object Viewer

In [24]:
%%bash
# copy pytorch files locally
git clone https://github.com/pytorch/serve.git \
  --branch=v0.3.0 \
  --depth=1

Cloning into 'serve'...
Note: checking out 'ccff97780628a01ab3b47d835d8e9dd01377d881'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [39]:
%%bash
#create dockerfile

cd serve/examples/image_classifier/mnist

cat > Dockerfile <<END
FROM pytorch/torchserve:0.3.0-cpu

COPY mnist.py mnist_cnn.pt mnist_handler.py /home/model-server/

USER root
RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
USER model-server
USER model-server

# expose health and prediction listener ports from the image
EXPOSE 7080
EXPOSE 7081

RUN torch-model-archiver   \
--model-name=mnist  \
 --version=1.0   \
 --model-file=/home/model-server/mnist.py   \
 --serialized-file=/home/model-server/mnist_cnn.pt   \
 --handler=/home/model-server/mnist_handler.py   \
 --export-path=/home/model-server/model-store

CMD ["torchserve","--start","--ts-config=/home/model-server/config.properties", "--models","mnist=mnist.mar","--model-store","/home/model-server/model-store"]

END

In [45]:
%%bash 
# create a cloud build deployment using docker

cd serve/examples/image_classifier/mnist

cat > cloudbuild.yaml <<END
steps:
# Build the container image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', '${IMAGE_NAME}', '.']
# Push the container image to Container Registry
- name: 'gcr.io/cloud-builders/docker'
  args: ['push', '${IMAGE_NAME}']
# Deploy container image to Cloud Run
- name: 'gcr.io/google.com/cloudsdktool/cloud-sdk'
  entrypoint: gcloud
  args: ['run', 'deploy', '${APP_NAME}','--port','7080','--image', '${IMAGE_NAME}', '--region', '${REGION}']
images:
- ${IMAGE_NAME}
END

gcloud builds submit > silent

Creating temporary tarball archive of 18 file(s) totalling 4.6 MiB before compression.
Uploading tarball of [.] to [gs://felipe-sandbox_cloudbuild/source/1633627899.037035-4d2c6cb22b5a4453b54ae5d19b2be2dc.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/felipe-sandbox/locations/global/builds/b0535341-904d-445d-b23f-9ff3fcb531d3].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/b0535341-904d-445d-b23f-9ff3fcb531d3?project=581970904807].
ERROR: (gcloud.builds.submit) build b0535341-904d-445d-b23f-9ff3fcb531d3 completed with status "FAILURE"


CalledProcessError: Command 'b"# create a cloud build deployment using docker\n\ncd serve/examples/image_classifier/mnist\n\ncat > cloudbuild.yaml <<END\nsteps:\n# Build the container image\n- name: 'gcr.io/cloud-builders/docker'\n  args: ['build', '-t', '${IMAGE_NAME}', '.']\n# Push the container image to Container Registry\n- name: 'gcr.io/cloud-builders/docker'\n  args: ['push', '${IMAGE_NAME}']\n# Deploy container image to Cloud Run\n- name: 'gcr.io/google.com/cloudsdktool/cloud-sdk'\n  entrypoint: gcloud\n  args: ['run', 'deploy', '${APP_NAME}','--port','7080','--image', '${IMAGE_NAME}', '--region', '${REGION}']\nimages:\n- ${IMAGE_NAME}\nEND\n\ngcloud builds submit > silent\n"' returned non-zero exit status 1.

In [ ]:
%%bash
ENDPOINT_ID="1648581346408267776"
PROJECT_ID="felipe-sandbox"

cat > INPUT-JSON <<END
{
  "model": {
    "displayName": ${APP_NAME},
    "predictSchemata": {},
    "containerSpec": {
      "imageUri": "gcr.io/${PROJECT_ID}/${APP_NAME}/serve-mnist",
      "ports": [
        {
          "containerPort": 7080
        }
      ],
      "predictRoute": "/predictions/mnist",
      "healthRoute": "/ping"
    },
    "artifactUri": "PATH_TO_MODEL_ARTIFACT_DIRECTORY",
  }
}

END

curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @INPUT-JSON \
"https://${REGION}-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/${REGION}/models:upload"

In [35]:
%%bash
ENDPOINT_ID="1648581346408267776"
PROJECT_ID="felipe-sandbox"
INPUT_DATA_FILE="INPUT-JSON"

cat > INPUT-JSON <<END
{
  "instances": [
    {
      "data": {
        "b64": "iVBORw0KGgoAAAANSUhEUgAAABwAAAAcCAAAAABXZoBIAAAAv0lEQVR4nGNgGKSA03faPyDwxibHu/7vvwfnzz/5tsgRU3LW33uukgwMCi1PdmBKOr7dAmEsuiiIKSssDpX8q4fbYYv/4ZZk3YTNWCg48HcGTrnOf39dcUgpzPv97+/b56LY5PKBIfTi+bt//7ptMSV7Py6NYWCQirn17zymJK8R1PRVd4RxuoqhG6erCEmevoBbbsqvUkxBXWMQabzk+wksOhZ9vHDh4oWPf1d6YZFUuff377+/9zp5cNtIHQAAtP5OgKw1m4AAAAAASUVORK5CYII="
      }
    }
  ]
}

END

curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-central1/endpoints/${ENDPOINT_ID}:predict \
-d "@${INPUT_DATA_FILE}"

{
  "predictions": [
    0,
    1
  ],
  "deployedModelId": "4502552892300853248",
  "model": "projects/581970904807/locations/us-central1/models/2825648126361075712",
  "modelDisplayName": "titanic_custom_training"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   369    0   218  100   151    158    109  0:00:01  0:00:01 --:--:--   268


In [23]:
!gcloud auth activate-service-account --key-file=/content/felipe-sandbox-ba2c6045e3e1.json

Activated service account credentials for: [pytorch-test@felipe-sandbox.iam.gserviceaccount.com]


In [ ]:
%%bash
docker build \
  --tag=us-central1-docker.pkg.dev/PROJECT_ID/getting-started-pytorch/serve-mnist \
  .